### 1. Setup

In [ ]:
import datetime
import requests
import pandas as pd
import numpy as np
from pathlib import Path

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('dark_background')


In [ ]:
SAVE_IMAGES = True

def f(valor, plus=False):
    if valor is None: return None
    valor = valor if type(valor) == int else float(valor)
    r = format(valor, ",").replace(".","!").replace(",",".").replace("!",",")
    return f"+{r}" if plus and valor > 0 else r


In [ ]:
# População residente em PT final 2019, via
# https://www.ine.pt/xportal/xmain?xpid=INE&xpgid=ine_indicadores&contecto=pi&indOcorrCod=0008273&selTab=tab0
# Coerente com a soma da população dos concelhos, vide POP_ARS abaixo
POP_PT = 10295909

# data_concelhos_new.csv:
# df[df.data == '11-11-2020'][["ars", "population"]].groupby('ars').sum()
POP_ARS = {
    'norte':   3568835,  # 3.57M
    'centro':  1650394,  # 1.65M
    'lvt':     3674534,  # 3.67M
    'alentejo': 466690,  # 0.46M
    'algarve':  438406,  # 0.44M
    'acores':   242796,  # 0.24M
    'madeira':  254254,  # 0.25M
}
POP_IDADE = {
    '0_9':     433332 + 461299,  #  0-04 + 05-09
    '10_19':   507646 + 549033,  # 10-14 + 15-19
    '20_29':   544575 + 547505,  # 20-24 + 25-29
    '30_39':   571355 + 679093,  # 30-34 + 35-39
    '40_49':   792670 + 782555,  # 40-44 + 45-49
    '50_59':   747581 + 734540,  # 50-54 + 55-59
    '60_69':   672758 + 620543,  # 60-64 + 65-69
    '70_79':   544016 + 429107,  # 70-74 + 75-79
    '80_plus': 352218 + 316442,  # 80-84 + 85 ou mais
}


### 2. Get Data

In [ ]:
# read data and reindex by date, for plots
data = pd.read_csv(Path.cwd() / '..' / '..' / 'data.csv', parse_dates=['data'], index_col='data', dayfirst=True)
data_amostras = pd.read_csv(Path.cwd() / '..' / '..' /'amostras.csv', parse_dates=['data'], index_col='data', dayfirst=True)
data_vacinas = pd.read_csv(Path.cwd() / '..' / '..' /'vacinas.csv', parse_dates=['data'], index_col='data', dayfirst=True)
data_vacinas_detalhe = pd.read_csv(Path.cwd() / '..' / '..' /'vacinas_detalhe.csv', parse_dates=['data'], index_col='data', dayfirst=True)

data = data.merge(data_amostras, how='outer', on=['data'])
data.tail(1).index, data_amostras.tail(1).index, data_vacinas.tail(1).index, data_vacinas_detalhe.tail(1).index


In [ ]:
# idosos

i=1
last_df = data.iloc[-i]
while np.isnan(last_df.obitos_0_9_m):
    i = i+1
    last_df = data.iloc[-i]
    print(last_df)
#print(data.columns)

idades = list(reversed(POP_IDADE.keys()))
for idade in idades:
    pop = POP_IDADE[idade]
    obitos = int(last_df[f"obitos_{idade}_m"] + last_df[f"obitos_{idade}_f"])
    confirmados = int(last_df[f"confirmados_{idade}_m"] + last_df[f"confirmados_{idade}_f"])
    recuperados = int(confirmados - obitos)
    print(f"idade={idade} pop={f(pop)} obitos={f(obitos)} ({f(round(100.0*obitos/pop, 3))}%) recuperados={f(recuperados)} ({f(round(100.0*recuperados/pop, 3))}%) confirmados={f(confirmados)} ({f(round(100.0*confirmados/pop, 3))}%)")
print()

last_df = data_vacinas_detalhe.iloc[-1]
for idade in reversed(['0_17', '18_24', '25_49', '50_64', '65_79','80+']):
    pop = int(last_df[f'populacao1_{idade}'])
    doses2 = int(last_df[f'doses2_{idade}'])
    doses1 = int(last_df[f'doses1_{idade}'])
    print(f"idade={idade} pop={f(pop)} doses2={f(doses2)} ({f(round(100.0*doses2/pop, 3))}%) doses1={f(doses1)} ({f(round(100.0*doses1/pop, 3))}%)")
print()


In [ ]:
df = data.copy()
df = df.iloc[-60:]
df['amostras7'] = df.amostras.diff(7).div(7)
df['amostras_pcr7'] = df.amostras_pcr.diff(7).div(7)
df['amostras_antigenio7'] = df.amostras_antigenio.diff(7).div(7)
df['confirmados7'] = df.confirmados.diff(7).div(7)

fig, axes = plt.subplots(figsize=(20,5))

axes.plot(df['amostras7'], lw=2, color="red", label='amostras total')
axes.plot(df['amostras_pcr7'], lw=2, color="orange", label='amostras pcr')
axes.plot(df['amostras_antigenio7'], lw=2, color="brown", label='amostras antigenio')
#last = float(df['amostras7'][-1])
#axes.axhline(linewidth=2, ls="--", color='white', y=last)
#axes.axhline(linewidth=2, ls="--", color='white', y=22000)
plt.ylabel('amostras', color="red")

axR = axes.twinx()
axR.plot(df['confirmados7'], lw=2, color="yellow", label='confirmados')
plt.ylabel('confirmados', color="yellow")

fig.legend()
axes.set_title(f'Amostras vs. Confirmados')
#plt.xticks(list(reversed(list(reversed(list(df.index)))[0::28])))
plt.xticks(list(reversed(list(reversed(list(df.index)))[0::7])))
plt.tight_layout()
if SAVE_IMAGES: plt.savefig(f"../../temp/diario_amostras_vs_confirmados.png")
plt.show()


In [ ]:
data['confirmados14'] = data.confirmados.diff(14)
data['incidencia'] = data.confirmados.diff(14) * 100000 / POP_PT
data[['confirmados14', 'incidencia']].tail()


### 3. Casos acumulados 90 dias
Asumindo as diretrizes da DGS de provavel imunidade de 90 dias, calcula os casos acumulados em 90 dias (total e 80+), os óbitos em 90 dias (total e 80+), portanto resultando aproximadamente nos recuperados e potenciais imunes para esse período.


In [ ]:
df = data.copy()

# Acumulado de confirmados durante 90 dias, total e 80+ (f+m), e percentagem entre ambos

# Cálculo confirmados 90 dias
df['confirmados_90d'] = df.confirmados.diff(90)
# Cálculo confirmados 90 dias para 80+
df['confirmados_80_plus_90d'] = (df.confirmados_80_plus_f + df.confirmados_80_plus_m).diff(90)
# Cálculo percentagem 80+ por total
# df['confirmados_perc'] = 100.0 * df.confirmados_80_plus_90d / df.confirmados_90d

# Acumulado de óbitos durante 90 dias, total e 80+ (f+m), e percentagem entre ambos

# Cálculo óbitos 90 dias
df['obitos_90d'] = df.obitos.diff(90)
# Cálculo óbitos 90 dias para 80+
df['obitos_80_plus_90d'] = (df.obitos_80_plus_f + df.obitos_80_plus_m).diff(90)
# Cálculo percentagem 80+ por total
# df['obitos_perc'] = 100.0 * df.obitos_80_plus_90d / df.obitos_90d

# Cálculo recuperados (confirmados menos óbitos) 90 dias
df['Recuperados 90d'] = df.confirmados_90d - df.obitos_90d
# Cálculo recupreados (confirmados menos óbitos) 90 dias para 80+
df['Recuperados 80+ 90d'] = df.confirmados_80_plus_90d - df.obitos_80_plus_90d

# Cálculo recuperados (confirmados menos óbitos) 90 dias
df['% Recuperados 90d'] = 100.0 * df['Recuperados 90d'] / POP_PT
# Cálculo recupreados (confirmados menos óbitos) 90 dias para 80+
df['% Recuperados 80+ 90d'] = 100 * df['Recuperados 80+ 90d'] / POP_IDADE['80_plus']


# Discard first 89 days
df = df[ df['confirmados_90d'].notnull() ]

df_90d = df


In [ ]:
df = df_90d # Ordem temporal inversa e mostra ultimos 7 dias
df[[
    'Recuperados 90d', 'Recuperados 80+ 90d',
    '% Recuperados 90d', '% Recuperados 80+ 90d',
    'confirmados_90d', 'confirmados_80_plus_90d', # 'confirmados_perc',
    'obitos_90d', 'obitos_80_plus_90d', # 'obitos_perc', 
    ]].iloc[::-1].head(7)


In [ ]:
df = df_90d
df[[
    'Recuperados 90d', 'Recuperados 80+ 90d',
    'confirmados_90d', 'confirmados_80_plus_90d', # 'confirmados_perc',
    'obitos_90d', 'obitos_80_plus_90d', # 'obitos_perc', 
    ]].plot(figsize=(20,5))
print()
noop = df[[
    # 'confirmados_perc', 'obitos_perc',
    '% Recuperados 90d', '% Recuperados 80+ 90d',
]].plot(figsize=(20,5))
plt.show()


### Incidência 120

In [ ]:
df = data.copy()

# 14 day accumulated, and per 100k and population
df['confirmados14'] = df.confirmados.diff(14)
df['incidencia'] = df.confirmados14 * 100_000 / POP_PT

for i in POP_ARS.keys():
  # per region, 14 day accumulated, and per 100k and population
  ii = i if i in ['acores', 'madeira'] else f'ars{i}'
  df[f'confirmados_{i}_14'] = df[f'confirmados_{ii}'].diff(14)
  df[f'incidencia_{i}'] = df[f'confirmados_{i}_14'] * 100_000 / POP_ARS[i]
  df[f'confirmados_{i}_7'] = df[f'confirmados_{ii}'].diff(7)
  df[f'incidencia_{i}_7'] = df[f'confirmados_{i}_7'] * 100_000 / POP_ARS[i]

for i in POP_IDADE.keys():
  # por idade, 14 day accumulated, and per 100k and population
  ii = i
  df[f'confirmados_{i}'] = df[f'confirmados_{ii}_m'] + df[f'confirmados_{ii}_f']
  df[f'confirmados_{i}_14'] = df[f'confirmados_{ii}'].diff(14)
  df[f'incidencia_{i}'] = df[f'confirmados_{i}_14'] * 100_000 / POP_IDADE[i]
  df[f'confirmados_{i}_7'] = df[f'confirmados_{ii}'].diff(7)
  df[f'incidencia_{i}_7'] = df[f'confirmados_{i}_7'] * 100_000 / POP_IDADE[i]

# discard first 13 days
df = df[ df.confirmados14.notnull() ]
df_incidencia = df


In [ ]:
def foo(k, n=None, df=df):
  fig, axes = plt.subplots(figsize=(20,5))

  axes.fill_between(df.index, 0, df[k], color="#592212", label=">960")
  axes.fill_between(df.index, 0, df[k], where=df[k] < 960, color="#700", label="480 a <960")
  axes.fill_between(df.index, 0, df[k], where=df[k] < 480, color="red", label="240 a <480")
  axes.fill_between(df.index, 0, df[k], where=df[k] < 240, color="orange", label="120 a <240")
  axes.fill_between(df.index, 0, df[k], where=df[k] < 120, color="yellow", label="60 a <120")
  axes.fill_between(df.index, 0, df[k], where=df[k] <  60, color="green", label="<60")

  axes.axhline(linewidth=2, ls="--", color='#592212', y=960)
  axes.axhline(linewidth=2, ls="--", color='#700', y=480)
  axes.axhline(linewidth=2, ls="--", color='red', y=240)
  axes.axhline(linewidth=2, ls="--", color='orange', y=120)
  axes.axhline(linewidth=2, ls="--", color='yellow', y=60)
  #axes.axhline(linewidth=2, ls="--", color='green', y=0)

  last = float(df[k][-1])
  axes.axhline(linewidth=2, ls="--", color='white', y=last)

  axes.legend()
  axes.set_title(f'Incidência {n or k}: {round(last, 1)}')
  plt.xticks(list(reversed(list(reversed(list(df.index)))[0::7])), rotation=90)
  plt.ylim(0, 210)
  plt.tight_layout()
  if SAVE_IMAGES: plt.savefig(f"../../temp/incidencia_{n or k}.png")
  plt.show()


In [ ]:
foo('incidencia', 'Nacional', df_incidencia)


In [ ]:
for k in POP_ARS.keys():
  foo(f'incidencia_{k}', k, df_incidencia)
  print()


In [ ]:
for k in POP_IDADE.keys():
  foo(f'incidencia_{k}', k, df_incidencia)
  print()


In [ ]:
df = df_incidencia
for k in POP_ARS.keys():
  mask = (df[f'incidencia_{k}'] >= 120)
  q = df[mask]
  q = q[[f'incidencia_{k}']]
  q['region'] = k
  print(q.head(1).append(q.tail(1)))


### FOOO

In [ ]:
# Quando é "há 14 dias"
df = data.copy()
df['past'] = df.data_dados.apply(lambda x: datetime.datetime.strptime(x, "%d-%m-%Y %H:%M") - datetime.timedelta(days=14))
df[ ['past'] ].tail(3)


In [ ]:
# Madeira
#datetime.datetime.today() - datetime.timedelta(days=14)
#(7382-6621) * 100_000 / 254254 # 03-17 304,4
#(7432-6680) * 100_000 / 254254 # 04-18 300,8
#(7478-6746) * 100_000 / 254254 # 05-19 292,8
#(7537-6799) * 100_000 / 254254 # 06-20 295,2
#(7570-6861) * 100_000 / 254254 # 07-03->21-02 # 279
(7613-6904) * 100_000 / 254254 # 08-03->22-02 # 279



In [ ]:
name=""
val1=-3023.0
val2=-829.0
# diff=-2194.0 perc=1.377848678213309

diff = val1 - val2

max(abs(val1), abs(val2))
#perc = max(abs(val1), abs(val2)) / abs(diff) if diff else 0
#print(f"name={name} val1={val1} val2={val2} diff={diff} perc={perc}")
#print( diff if perc > 5 else 0 )



In [ ]:
#df.confirmados.diff(14)
#810094 - 12569.0 # 797525
#df.confirmados_novos.tail(14).head(14).sum()
df.tail(15).head(1)
# 07 -> 21


In [ ]:
data[ [ x for x in data.columns if x.startswith("obitos_") and x[-1] in ["f","m"]]].diff(1).tail(2)


In [ ]:
#data.columns
data.head()

In [ ]:
df = data.copy()

df['confirmados_1'] = df['confirmados'].diff(1).div(1)
df['confirmados_7'] = df['confirmados'].diff(7).div(7)
df['confirmados_14'] = df['confirmados'].diff(14).div(14)

df['confirmados_segunda_derivada_1'] = df['confirmados_1'].div(df['confirmados_1'].shift(1))
df['confirmados_segunda_derivada_7'] = df['confirmados_7'].div(df['confirmados_7'].shift(1))
df['confirmados_segunda_derivada_14'] = df['confirmados_14'].div(df['confirmados_14'].shift(1))

df = df.iloc[-15:]
axes = df[[
    'confirmados_segunda_derivada_1',
    'confirmados_segunda_derivada_7',
    'confirmados_segunda_derivada_14',
    ]].plot(kind='line', title='Variação confirmados "Rt" média 7 dias', linestyle='--', marker='o', rot=0, fontsize=8, figsize=(20,5))
axes.axhline(linewidth=1, color='grey', y=1, lw=2)
axes.axhline(linewidth=1, color='yellow', y=0.75)
axes.axhline(linewidth=1, color='green', y=0.50)
axes.axhline(linewidth=1, color='orange', y=1.25)
axes.axhline(linewidth=1, color='red', y=1.50)
plt.ylim(0.25, 1.75)
plt.show()


In [ ]:
df = data.copy()

cols = [x for x in df.columns if x.startswith("confirmados_") and x != 'confirmados_novos']
for col in cols:
    df[f'{col}_7'] = df[col].diff(7).div(7)
    df[f'{col}_7_1'] = df[f'{col}_7'].diff()

cols2 = [f'{col}_7_1' for col in cols]
foo = df[cols2].tail(1).transpose()
foo.columns = ['foo']
foo = foo[ foo.foo > 0 ] #.sort_values('foo')
foo = foo.transpose()

df = df.iloc[-14:]
axes = df[
    cols2
    #[col for col in foo.columns if col != 'confirmados_madeira_7_1']
].plot(kind='line', title='Confirmados média 7 dias', linestyle='--', marker='o', rot=0, fontsize=8, figsize=(20,10))
axes.axhline(linewidth=1, color='grey')
plt.show()


In [ ]:
# Ativos going up

df = data.copy()

df['ativos7'] = df.ativos.diff(7)
df['ativos1'] = df.ativos.diff(1)
df['recuperados7'] = df.recuperados.diff(7)
df['recuperados1'] = df.recuperados.diff(1)
df['obitos7'] = df.obitos.diff(7)
df['obitos1'] = df.obitos.diff(1)
df = df.iloc[-60:]
df[[
    #'obitos7', 'obitos1',
    'ativos7', # 'ativos1', 
    'recuperados7', # 'recuperados1'
    ]].plot(kind='line', title='Ativos', linestyle='--', marker='o', rot=0, fontsize=8, figsize=(20,5))
plt.show()


In [ ]:
df = data.copy()

df['internados14'] = df.internados.diff(14).div(14)
df['internados7'] = df.internados.diff(7).div(7)
df['internados1'] = df.internados.diff(1)
df['internados7_diff'] = df.internados7.diff(1)
df['internados_uci14'] = df.internados_uci.diff(14).div(14)
df['internados_uci7'] = df.internados_uci.diff(7).div(7)
df['internados_uci1'] = df.internados_uci.diff(1)
df['internados_uci7_diff'] = df.internados_uci7.diff(1)
df['internados_enfermaria14'] = df.internados_enfermaria.diff(14).div(14)
df['internados_enfermaria7'] = df.internados_enfermaria.diff(7).div(7)
df['internados_enfermaria7_diff'] = df.internados_enfermaria7.diff(1)
df['internados_enfermaria1'] = df.internados_enfermaria.diff(1)
#df = df.iloc[-90:]
df = df.iloc[-28:]


In [ ]:
last1, last7 = round(float(df['internados_enfermaria1'][-1]), 1), round(float(df['internados_enfermaria7'][-1]), 1)
axes = df[[
    'internados_enfermaria7',
    'internados_enfermaria1',
    'internados_enfermaria7_diff',
    'internados_enfermaria14',
    #'internados_enfermaria',
    ]].plot(kind='line', title=f'Internados Enfermaria, 1 ({last1}) e 7 ({last7}) dias', linestyle='--', marker='o', rot=0, fontsize=8, figsize=(20,5))
axes.axhline(linewidth=1, color='red', y=0)
axes.legend(loc="lower left")
plt.ylim(-20, 10)
plt.tight_layout()
if SAVE_IMAGES: plt.savefig(f"../../temp/variacao_enfermaria.png")
plt.show()


In [ ]:
last1, last7 = round(float(df['internados_uci1'][-1]), 1), round(float(df['internados_uci7'][-1]), 1)
axes = df[[
    'internados_uci7',
    'internados_uci1',
    'internados_uci7_diff',
    'internados_uci14',
    #'internados_uci',
    ]].plot(kind='line', title=f'Internados UCI, 1 ({last1}) e 7 ({last7}) dias', linestyle='--', marker='o', rot=0, fontsize=8, figsize=(20,5))
axes.axhline(linewidth=1, color='red', y=0)
axes.legend(loc="lower left")
plt.ylim(-15, 10)
plt.tight_layout()
if SAVE_IMAGES: plt.savefig(f"../../temp/variacao_uci.png")
plt.show()


### ---

In [ ]:
# passado melhor que hoje
df = data.copy()
df['confirmados7'] = df.confirmados.diff(7).div(7)
df['obitos7'] = df.obitos.diff(7).div(7)
df['confirmados1'] = df.confirmados.diff(1).div(1)
df['obitos1'] = df.obitos.diff(1).div(1)

last = df.tail(1)[['confirmados1', 'confirmados7', 'obitos1', 'obitos7']]
last

In [ ]:
last_confirmados = int(last['confirmados7'])
df['melhor_que_hoje'] = df.confirmados7 * (df.confirmados7 < last_confirmados)
df[['melhor_que_hoje']].plot(figsize=(15,4), title=f'Dias com confirmados melhor que hoje, média {last_confirmados}')


In [ ]:
last_obitos = int(last['obitos7'])
df['melhor_que_hoje'] = df.obitos7 * (df.obitos7 < last_obitos)
df[['melhor_que_hoje']].plot(figsize=(15,4), title=f'Dias com óbitos melhor que hoje (<{last_obitos})')


In [ ]:
last_internados = int(df.tail(1).internados)
df['melhor_que_hoje'] = df.internados * (df.internados < last_internados)
df[['melhor_que_hoje']].plot(figsize=(15,4), title=f'Dias com internados melhor que hoje (<{last_internados})')
plt.show()


In [ ]:
last_internados_enfermaria = int(df.tail(1).internados_enfermaria)
df['melhor_que_hoje'] = df.internados_enfermaria * (df.internados_enfermaria < last_internados_enfermaria)
df[['melhor_que_hoje']].plot(figsize=(15,4), title=f'Dias com internados enfermaria melhor que hoje (<{last_internados_enfermaria})')
plt.show()


In [ ]:
last_internados_uci = int(df.tail(1).internados_uci)
df['melhor_que_hoje'] = df.internados_uci * (df.internados_uci < last_internados_uci)
df[['melhor_que_hoje']].plot(figsize=(15,4), title=f'Dias com internados UCI melhor que hoje (<{last_internados_uci})')
plt.show()


In [ ]:
#----

In [ ]:
df = data.copy()
df['confirmados1'] = df.confirmados.diff(1).div(1)
df['obitos1'] = df.obitos.diff(1).div(1)
df['confirmados7'] = df.confirmados.diff(7).div(7)
df['obitos7'] = df.obitos.diff(7).div(7)


In [ ]:
df[ df.index == '2020-04-01' ][['confirmados1', 'confirmados7', 'obitos1', 'obitos7', 'internados', 'internados_uci']]


In [ ]:
df[ df.index == '2020-05-01' ][['confirmados1', 'confirmados7', 'obitos1', 'obitos7', 'internados', 'internados_uci']]


In [ ]:
df[ df.index == '2020-06-01' ][['confirmados1', 'confirmados7', 'obitos1', 'obitos7', 'internados', 'internados_uci']]


In [ ]:
df = data.copy()
df['confirmados7'] = df.confirmados.diff(7).div(7)
df['obitos7'] = df.obitos.diff(7).div(7)
df['confirmados1'] = df.confirmados.diff(1).div(1)
df['obitos1'] = df.obitos.diff(1).div(1)
df.tail(1)[['confirmados7', 'confirmados1', 'obitos7', 'obitos1']]


In [ ]:
flatten = lambda t: [item for sublist in t for item in sublist]
idades = ['0_9', '10_19', '20_29', '30_39', '40_49', '50_59', '60_69', '70_79', '80_plus']

if False:
    df = data.copy()
    for k in idades:
        df[f'confirmados_{k}'] = df[f'confirmados_{k}_f'] + df[f'confirmados_{k}_m']
    cols = [f'confirmados_{k}' for k in idades]

    df[cols] # .diff(14).tail(30)


In [ ]:
#df_amostras.tail(30)['amostras_novas'].plot(kind='line', title='Ativos', linestyle='--', marker='o', rot=0, fontsize=8, figsize=(15,4))
#df_amostras['amostras7']=df_amostras.amostras.diff(7)
#df_amostras.tail(10)

#df['confirmados14']=df.confirmados.diff(14)
#df.confirmados14.tail(14) * 100_000 / 10295909


In [ ]:
df = data.copy()

df['confirmados_madeira14'] = df.confirmados_madeira.diff(14).div(14)
df['confirmados_madeira7'] = df.confirmados_madeira.diff(7).div(7)
df['confirmados_madeira1'] = df.confirmados_madeira.diff(1)
df = df.iloc[-60:]
df[[
    'confirmados_madeira1',
    'confirmados_madeira7',
    #'confirmados_madeira14',
]].plot(kind='line', title='Confirmados Madeira, diários, média 7 e 14 dias', linestyle='--', marker='o', rot=0, fontsize=8, figsize=(15,4))
plt.show()


In [ ]:
df.confirmados_madeira1.tail(10)


In [ ]:
df = data.copy()

df['confirmados7'] = df.confirmados.diff(7).div(7)
df['confirmados1'] = df.confirmados.diff(1)
df = df.iloc[-60:]
df[['confirmados1', 'confirmados7']].plot(kind='line', title='Confirmados', linestyle='--', marker='o', rot=0, fontsize=8, figsize=(15,4))
plt.show()


In [ ]:
df = data.copy()
df['dom-sab'] = df.data_dados.apply(lambda x: datetime.datetime.strptime(x, "%d-%m-%Y %H:%M").weekday() * 100 / 6)

df['internados_enfermaria1'] = df.internados.diff(1)
df = df.iloc[-60:]
axes = df[['dom-sab', 'internados_enfermaria1']].plot(kind='line', title='Enfermaria', linestyle='--', marker='o', rot=0, fontsize=8, figsize=(15,4))
axes.axhline(linewidth=1, color='grey')
plt.show()


In [ ]:

(df['confirmados'] / df['amostras']).tail(10)


In [ ]:
# group by
flatten = lambda t: [item for sublist in t for item in sublist]
idades = ['0_9', '10_19', '20_29', '30_39', '40_49', '50_59', '60_69', '70_79', '80_plus']


In [ ]:
df = data.copy()

cols_obitos_lt50 = flatten([ [f"obitos_{x}_f", f"obitos_{x}_m"] for x in idades[0:5]])
obitos_lt50 = int(df.loc[:, cols_obitos_lt50].diff(1).sum(axis=1).tail(1))
obitos_lt50_7d = int(df.loc[:, cols_obitos_lt50].diff(7).sum(axis=1).tail(1))

obitos_lt50, obitos_lt50_7d


In [ ]:
df = data.copy()

o40 = flatten([ [f"obitos_{x}_f", f"obitos_{x}_m"] for x in idades[0:5]])
# print(o40)
o = df.loc[:, o40].diff(1).sum(axis=1).tail(1)
print(f"obitos <= 49 = {int(o)}")

p60 = flatten([ [f"confirmados_{x}_f", f"confirmados_{x}_m"] for x in idades[6:]])
m60 = flatten([ [f"confirmados_{x}_f", f"confirmados_{x}_m"] for x in idades[:6]])
# print(m60, p60)

a=df.loc[:, p60].diff(1).sum(axis=1).tail(1)
print(int(a))
b=df.loc[:, m60].diff(1).sum(axis=1).tail(1)
print(int(b))
print(int(a+b))
print(int(df.tail(1).confirmados_novos))

In [ ]:
#DATA_FILEPATH = Path.cwd() / '..' / 'data_concelhos_new.csv'
#data = pd.read_csv(DATA_FILEPATH)

#df = data.copy()
#df = df[df.data == '11-11-2020'][["ars", "population"]]
#df.groupby('ars').sum()


In [ ]:
df = data.copy()
df['confirmados_14'] = df.confirmados.diff(14)
#df.tail(1).confirmados_14 * 100000 / 10295909
df[ df.index == '2021-02-16' ].confirmados_14 * 100000 / 10295909


In [ ]:
# idades
df = data.copy()
df['confirmados_0_9'] = df['confirmados_0_9_m'] + df['confirmados_0_9_f']
df['confirmados_10_19'] = df['confirmados_10_19_m'] + df['confirmados_10_19_f']
df['confirmados_20_29'] = df['confirmados_20_29_m'] + df['confirmados_20_29_f']
df['confirmados_30_39'] = df['confirmados_30_39_m'] + df['confirmados_30_39_f']
#cols = ['confirmados_0_9_m', 'confirmados_0_9_f', 'confirmados_10_19_m', 'confirmados_10_19_f']
cols = ['confirmados_0_9', 'confirmados_10_19', 'confirmados_20_29', 'confirmados_30_39']
for col in cols:
    df[f"{col}_1"] = df[col].diff(1)
    df[f"{col}_7"] = df[col].diff(7).div(7)
    df[f"{col}_7_1"] = df[f"{col}_7"].diff(1)

df = df.iloc[-60:]
cols2 = [f"{col}_7" for col in cols]
#cols2 = [f"{col}_7_1" for col in cols]
axes = df[cols2].plot(figsize=(20,4))
#plt.ylim(0, 100)
plt.ylim(-10, 100)
plt.axhline(y=0)
plt.show()


In [ ]:
#cols2 = [f"{col}_1" for col in cols]
cols2 = [f"{col}_7" for col in cols]
df[cols2].tail(15)

